In [ ]:
'''
La derniere étape de notre projet est d'utiliser les modèles de machine learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel.
'''

In [2]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Copie du jeux de données.
df_2022_continents = df_2022.copy()

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
X_test2022_continents=df_2022_continents.drop(['Life Ladder', 'Country name'],axis=1)
y_test2022=df_2022_continents['Life Ladder']

#On encode les continents de X_2022
X_test2022_continents=pd.get_dummies(X_test2022_continents,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
X_test2022_continents[cols]=scaler.fit_transform(X_test2022_continents[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
X_test2022_continents=X_test2022_continents.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022_continents et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [3]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
y_train2021=y_train2021['Life Ladder'].values

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])

In [4]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model_continents = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model_continents.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model_continents.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model_continents.score(X_test2022_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.985415891456359
Score sur jeu de test 2022 : 0.7969897709944447


In [5]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_continents = rf_model_continents.predict(X_test2022_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_continents=np.round(y_predic2022_continents,2)

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_continents['Predictions']=y_predic2022_continents
df_2022_continents=df_2022_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_continents['Ecart en %']=np.round((df_2022_continents['Predictions']-df_2022_continents['Life Ladder'])/df_2022_continents['Life Ladder']*100,1)

In [6]:
df_2022_continents.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.77,-3.3
6,Bangladesh,South Asia,3.41,3.98,16.7
7,Belgium,Western Europe,6.86,6.99,1.9
8,Benin,Sub-Saharan Africa,4.22,3.97,-5.9
9,Bolivia,Latin America and Caribbean,5.93,5.38,-9.3
10,Botswana,Sub-Saharan Africa,3.44,4.57,32.8


In [7]:
df_2022_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

# On constate que le modèle prédit plutot bien avec toutefois des écarts plus importants pour les valeurs extremes.

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.775556,-6.844444
Southeast Asia,5.530000,5.115000,-6.750000
East Asia,5.973333,5.580000,-6.700000
Commonwealth of Independent States,5.592500,5.206250,-6.625000
Central and Eastern Europe,6.044545,5.749091,-4.618182
North America and ANZ,6.907500,6.782500,-1.875000
Western Europe,6.893333,6.807222,-1.300000
South Asia,4.270000,4.093333,-0.900000
Sub-Saharan Africa,4.289643,4.251429,1.678571


In [8]:
display(df_2022_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.94,67.7
93,Sierra Leone,Sub-Saharan Africa,2.56,3.61,41.0
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.13,28.7
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0
6,Bangladesh,South Asia,3.41,3.98,16.7
10,Botswana,Sub-Saharan Africa,3.44,4.57,32.8
19,Comoros,Sub-Saharan Africa,3.55,3.86,8.7
101,Tanzania,Sub-Saharan Africa,3.62,4.04,11.6
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.97,-15.2
22,Costa Rica,Latin America and Caribbean,7.08,6.65,-6.1
64,Luxembourg,Western Europe,7.23,6.90,-4.6
83,Norway,Western Europe,7.30,7.44,1.9
78,Netherlands,Western Europe,7.39,7.06,-4.5
98,Sweden,Western Europe,7.43,7.47,0.5
45,Iceland,Western Europe,7.45,7.39,-0.8
26,Denmark,Western Europe,7.55,7.44,-1.5
50,Israel,Middle East and North Africa,7.66,6.81,-11.1
33,Finland,Western Europe,7.73,7.51,-2.8


In [9]:
display(df_2022_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.94,67.7
93,Sierra Leone,Sub-Saharan Africa,2.56,3.61,41.0
10,Botswana,Sub-Saharan Africa,3.44,4.57,32.8
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.13,28.7
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4
6,Bangladesh,South Asia,3.41,3.98,16.7
101,Tanzania,Sub-Saharan Africa,3.62,4.04,11.6
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0
39,Ghana,Sub-Saharan Africa,4.19,4.58,9.3


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.32,-25.6
42,Guinea,Sub-Saharan Africa,5.32,4.11,-22.7
77,Nepal,South Asia,5.47,4.27,-21.9
80,Nicaragua,Latin America and Caribbean,6.39,5.32,-16.7
30,El Salvador,Latin America and Caribbean,6.49,5.41,-16.6
71,Mexico,Latin America and Caribbean,7.04,5.89,-16.3
92,Senegal,Sub-Saharan Africa,4.91,4.13,-15.9
63,Lithuania,Central and Eastern Europe,7.04,5.97,-15.2
112,Vietnam,Southeast Asia,6.27,5.32,-15.2
73,Mongolia,East Asia,5.79,4.92,-15.0


In [ ]:
'''
On pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele 
vers une réponse plus cohérente en fonction du continents. 
On réalise donc un essai sur les mêmes données amputées de la variable continent.
'''

In [11]:
# Séparation des variables explicatives de la variable cible et suppression des noms de pays.
X_test2022_no_continents=df_2022.drop(['Life Ladder','Country name','Regional indicator'],axis=1)
y_test2022_no_continents=df_2022['Life Ladder']

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022_no_continents[cols]=scaler.fit_transform(X_test2022_no_continents[cols])

#X_test2022_no_continents et y_test2022_no_continents sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test

In [12]:
# Suppression variables des continents du jeu de données 2021.
X_train2021_no_continents=X_train2021.drop(["Regional indicator_Commonwealth of Independent States"
                              ,"Regional indicator_East Asia"
                              ,"Regional indicator_Latin America and Caribbean"
                              ,"Regional indicator_Middle East and North Africa"
                              ,"Regional indicator_North America and ANZ"
                              ,"Regional indicator_South Asia"
                              ,"Regional indicator_Southeast Asia"
                              ,"Regional indicator_Sub-Saharan Africa"
                              ,"Regional indicator_Western Europe"],axis=1)

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021_no_continents[cols]=scaler.fit_transform(X_train2021_no_continents[cols])

In [13]:
from sklearn.ensemble import RandomForestRegressor
rf_model_no_continents = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model_no_continents.fit(X_train2021_no_continents,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model_no_continents.score(X_train2021_no_continents,y_train2021))
print("Score sur jeu de test 2022 :",rf_model_no_continents.score(X_test2022_no_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9834432844119608
Score sur jeu de test 2022 : 0.7378728370603596


In [14]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_no_continents = rf_model_no_continents.predict(X_test2022_no_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_no_continents=np.round(y_predic2022_no_continents,2)

# Copie du jeu de donnée 2022 et suppression des continents.
df_2022_no_continents = df_2022.copy()

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_no_continents['Predictions']=y_predic2022_no_continents
df_2022_no_continents=df_2022_no_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_no_continents['Ecart en %']=np.round((df_2022_no_continents['Predictions']-df_2022_no_continents['Life Ladder'])/df_2022_no_continents['Life Ladder']*100,1)

In [15]:
df_2022_no_continents

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.02,-3.8
3,Armenia,Commonwealth of Independent States,5.38,4.91,-8.7
4,Australia,North America and ANZ,7.04,6.89,-2.1
5,Austria,Western Europe,7.00,6.67,-4.7
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.08,-9.1
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.23,-13.1
112,Vietnam,Southeast Asia,6.27,5.31,-15.3


In [16]:
df_2022_no_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.533889,-10.800000
Commonwealth of Independent States,5.592500,5.076250,-8.975000
Southeast Asia,5.530000,5.105000,-6.800000
East Asia,5.973333,5.613333,-6.166667
Central and Eastern Europe,6.044545,5.780909,-4.063636
North America and ANZ,6.907500,6.650000,-3.775000
Western Europe,6.893333,6.761111,-1.905556
South Asia,4.270000,4.120000,0.066667
Sub-Saharan Africa,4.289643,4.248929,1.467857


In [17]:
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.97,68.9
93,Sierra Leone,Sub-Saharan Africa,2.56,3.46,35.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.04,25.9
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
66,Malawi,Sub-Saharan Africa,3.36,3.68,9.5
6,Bangladesh,South Asia,3.41,3.97,16.4
10,Botswana,Sub-Saharan Africa,3.44,4.60,33.7
19,Comoros,Sub-Saharan Africa,3.55,3.90,9.9
101,Tanzania,Sub-Saharan Africa,3.62,4.27,18.0
32,Ethiopia,Sub-Saharan Africa,3.63,4.27,17.6


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.95,-15.5
22,Costa Rica,Latin America and Caribbean,7.08,6.61,-6.6
64,Luxembourg,Western Europe,7.23,6.83,-5.5
83,Norway,Western Europe,7.30,7.39,1.2
78,Netherlands,Western Europe,7.39,7.05,-4.6
98,Sweden,Western Europe,7.43,7.33,-1.3
45,Iceland,Western Europe,7.45,7.41,-0.5
26,Denmark,Western Europe,7.55,7.29,-3.4
50,Israel,Middle East and North Africa,7.66,6.55,-14.5
33,Finland,Western Europe,7.73,7.37,-4.7


In [18]:
display(df_2022_no_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_no_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.97,68.9
93,Sierra Leone,Sub-Saharan Africa,2.56,3.46,35.2
10,Botswana,Sub-Saharan Africa,3.44,4.60,33.7
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.04,25.9
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
101,Tanzania,Sub-Saharan Africa,3.62,4.27,18.0
32,Ethiopia,Sub-Saharan Africa,3.63,4.27,17.6
6,Bangladesh,South Asia,3.41,3.97,16.4
39,Ghana,Sub-Saharan Africa,4.19,4.64,10.7
19,Comoros,Sub-Saharan Africa,3.55,3.90,9.9


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.33,-25.5
77,Nepal,South Asia,5.47,4.15,-24.1
71,Mexico,Latin America and Caribbean,7.04,5.38,-23.6
30,El Salvador,Latin America and Caribbean,6.49,5.01,-22.8
42,Guinea,Sub-Saharan Africa,5.32,4.15,-22.0
80,Nicaragua,Latin America and Caribbean,6.39,5.00,-21.8
43,Honduras,Latin America and Caribbean,5.93,4.74,-20.1
41,Guatemala,Latin America and Caribbean,6.15,4.95,-19.5
37,Georgia,Commonwealth of Independent States,5.29,4.30,-18.7
9,Bolivia,Latin America and Caribbean,5.93,4.88,-17.7


In [19]:
# Pour terminer, on aggrège les deux jeux de données.
df_2022 = df_2022_continents
df_2022

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.77,-3.3
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5
110,Uzbekistan,Commonwealth of Independent States,6.02,5.38,-10.6
112,Vietnam,Southeast Asia,6.27,5.32,-15.2


In [20]:
# Changement de nom des colonnes
new_names =  {'Predictions' : 'Prédictions avec continents',
              'Ecart en %'  : 'Écart en % avec continents'}

df_2022 = df_2022.rename(new_names, axis = 1)
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.01,-0.4
5,Austria,Western Europe,7.00,6.77,-3.3
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5
110,Uzbekistan,Commonwealth of Independent States,6.02,5.38,-10.6
112,Vietnam,Southeast Asia,6.27,5.32,-15.2


In [21]:
df_2022['Prédictions sans continents'] = df_2022_no_continents['Predictions']
df_2022['Écart en % sans continents'] = df_2022_no_continents['Ecart en %']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2,6.02,-3.8
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1
5,Austria,Western Europe,7.00,6.77,-3.3,6.67,-4.7
...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.08,-9.1
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.38,-10.6,5.23,-13.1
112,Vietnam,Southeast Asia,6.27,5.32,-15.2,5.31,-15.3


In [22]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['Différences valeurs écarts'] = df_2022['Prédictions sans continents']-df_2022['Prédictions avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2,0.02
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2,6.02,-3.8,-0.10
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7,-0.08
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1,-0.12
5,Austria,Western Europe,7.00,6.77,-3.3,6.67,-4.7,-0.10
...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.08,-9.1,-0.04
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5,6.33,-5.1,0.03
110,Uzbekistan,Commonwealth of Independent States,6.02,5.38,-10.6,5.23,-13.1,-0.15
112,Vietnam,Southeast Asia,6.27,5.32,-15.2,5.31,-15.3,-0.01


In [23]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['% variations écarts'] = df_2022['Écart en % sans continents']-df_2022['Écart en % avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2,0.02,0.4
2,Argentina,Latin America and Caribbean,6.26,6.12,-2.2,6.02,-3.8,-0.10,-1.6
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7,-0.08,-1.5
4,Australia,North America and ANZ,7.04,7.01,-0.4,6.89,-2.1,-0.12,-1.7
5,Austria,Western Europe,7.00,6.77,-3.3,6.67,-4.7,-0.10,-1.4
...,...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.12,-8.5,6.08,-9.1,-0.04,-0.6
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5,6.33,-5.1,0.03,0.4
110,Uzbekistan,Commonwealth of Independent States,6.02,5.38,-10.6,5.23,-13.1,-0.15,-2.5
112,Vietnam,Southeast Asia,6.27,5.32,-15.2,5.31,-15.3,-0.01,-0.1


In [24]:
df_2022.describe()

,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,5.591359,5.396796,-1.989320,5.334951,-2.979612,-0.061845,-0.990291
std,1.229449,1.069412,12.599968,1.052056,13.251788,0.151708,2.724310
min,2.350000,3.610000,-25.600000,3.460000,-25.500000,-0.590000,-10.700000
25%,4.675000,4.450000,-8.450000,4.350000,-10.050000,-0.140000,-2.100000
50%,5.890000,5.410000,-3.300000,5.240000,-4.700000,-0.040000,-0.600000
75%,6.465000,6.135000,1.200000,6.110000,1.100000,0.030000,0.450000
max,7.730000,7.510000,67.700000,7.410000,68.900000,0.350000,6.400000


In [25]:
# On constate que les prédictions sont moins précises si l'on enlève les données relatives aux continents.

In [26]:
# Séparation des écarts en positifs et négatifs
ecarts_positifs = df_2022[df_2022['Écart en % avec continents'] > 0]['Écart en % avec continents']
ecarts_negatifs = df_2022[df_2022['Écart en % avec continents'] < 0]['Écart en % avec continents']

# Calcul de la moyenne et de la médiane pour les écarts positifs
ecart_positif_moyen = ecarts_positifs.mean()
ecart_positif_median = ecarts_positifs.median()

# Calcul de la moyenne et de la médiane pour les écarts négatifs
ecart_negatif_moyen = ecarts_negatifs.mean()
ecart_negatif_median = ecarts_negatifs.median()

# Affichage des résultats
print("Écart négatif moyen:", ecart_negatif_moyen)
print("Écart positif moyen:", ecart_positif_moyen)
print("Écart négatif médian:", ecart_negatif_median)
print("Écart positif médian:", ecart_positif_median)

Écart négatif moyen: -7.640845070422537
Écart positif moyen: 10.890322580645162
Écart négatif médian: -6.4
Écart positif médian: 5.6


In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE)
# avzec les continents.
mse_continents = mean_squared_error(y_test2022, y_predic2022_continents)
rmse_continents = np.sqrt(mse_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne avec les continents (MSE) : {mse_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : {rmse_continents}")

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE) 
# sans les continents.
mse_no_continents = mean_squared_error(y_test2022, y_predic2022_no_continents)
rmse_no_continents = np.sqrt(mse_no_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne sans les continents (MSE) : {mse_no_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : {rmse_no_continents}")

Erreur quadratique moyenne avec les continents (MSE) : 0.3037592233009709
Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : 0.5511435596112604
Erreur quadratique moyenne sans les continents (MSE) : 0.3927893203883494
Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : 0.6267290645792242


In [28]:
# On constate que la RMSE augmente sans les continents. Le modèles est donc bien moins performant.

In [29]:
from joblib import dump, load

dump(rf_model_continents, '../models/happinessmeter.joblib')

['../models/happinessmeter.joblib']